Copyright (c) Microsoft Corporation. All rights reserved.  

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/onnx/onnx-convert-aml-deploy-tinyyolo.png)

# YOLO Real-time Object Detection using ONNX on AzureML

This example shows how to convert the TinyYOLO model from CoreML to ONNX and operationalize it as a web service using Azure Machine Learning services and the ONNX Runtime.

## What is ONNX
ONNX is an open format for representing machine learning and deep learning models. ONNX enables open and interoperable AI by enabling data scientists and developers to use the tools of their choice without worrying about lock-in and flexibility to deploy to a variety of platforms. ONNX is developed and supported by a community of partners including Microsoft, Facebook, and Amazon. For more information, explore the [ONNX website](http://onnx.ai).

## YOLO Details
You Only Look Once (YOLO) is a state-of-the-art, real-time object detection system. For more information about YOLO, please visit the [YOLO website](https://pjreddie.com/darknet/yolo/).

## Prerequisites

To make the best use of your time, make sure you have done the following:

* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration](../../../configuration.ipynb) notebook to:
    * install the AML SDK
    * create a workspace and its configuration file (config.json)

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

#### Install necessary packages

You'll need to run the following commands to use this tutorial:

```sh
pip install onnxmltools
pip install coremltools  # use this on Linux and Mac
pip install git+https://github.com/apple/coremltools  # use this on Windows
```

## Convert model to ONNX

First we download the CoreML model. We use the CoreML model from [Matthijs Hollemans's tutorial](https://github.com/hollance/YOLO-CoreML-MPSNNGraph). This may take a few minutes.

In [ ]:
import urllib.request

coreml_model_url = "https://github.com/hollance/YOLO-CoreML-MPSNNGraph/raw/master/TinyYOLO-CoreML/TinyYOLO-CoreML/TinyYOLO.mlmodel"
urllib.request.urlretrieve(coreml_model_url, filename="TinyYOLO.mlmodel")


Then we use ONNXMLTools to convert the model.

In [ ]:
import onnxmltools
import coremltools

# Load a CoreML model
coreml_model = coremltools.utils.load_spec('TinyYOLO.mlmodel')

# Convert from CoreML into ONNX
onnx_model = onnxmltools.convert_coreml(coreml_model, 'TinyYOLOv2')

# Save ONNX model
onnxmltools.utils.save_model(onnx_model, 'tinyyolov2.onnx')

import os
print(os.path.getsize('tinyyolov2.onnx'))

## Deploying as a web service with Azure ML

### Load Azure ML workspace

We begin by instantiating a workspace object from the existing workspace created earlier in the configuration notebook.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = '\n')

### Registering your model with Azure ML

Now we upload the model and register it in the workspace.

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = "tinyyolov2.onnx",
                       model_name = "tinyyolov2",
                       tags = {"onnx": "demo"},
                       description = "TinyYOLO",
                       workspace = ws)

#### Displaying your registered models

You can optionally list out all the models that you have registered in this workspace.

In [ ]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

### Write scoring file

We are now going to deploy our ONNX model on Azure ML using the ONNX Runtime. We begin by writing a score.py file that will be invoked by the web service call. The `init()` function is called once when the container is started so we load the model using the ONNX Runtime into a global session object.

In [ ]:
%%writefile score.py
import json
import time
import sys
import os
from azureml.core.model import Model
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime

def init():
    global session
    model = Model.get_model_path(model_name = 'tinyyolov2')
    session = onnxruntime.InferenceSession(model)

def preprocess(input_data_json):
    # convert the JSON data into the tensor input
    return np.array(json.loads(input_data_json)['data']).astype('float32')

def postprocess(result):
    return np.array(result).tolist()

def run(input_data_json):
    try:
        start = time.time()   # start timer
        input_data = preprocess(input_data_json)
        input_name = session.get_inputs()[0].name  # get the id of the first input of the model   
        result = session.run([], {input_name: input_data})
        end = time.time()     # stop timer
        return {"result": postprocess(result),
                "time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}

### Setting up inference configuration
First we create a YAML file that specifies which dependencies we would like to see in our container.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=["numpy","onnxruntime==0.4.0","azureml-core"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Then we create the inference configuration.

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="myenv.yml",
                                   extra_docker_file_steps = "Dockerfile")

### Deploy the model

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'web service for TinyYOLO ONNX model')

The following cell will take a few minutes to run as the model gets packaged up and deployed to ACI.

In [ ]:
aci_service_name = 'my-aci-service-15ad'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In case the deployment fails, you can check the logs. Make sure to delete your aci_service before trying again.

In [ ]:
if aci_service.state != 'Healthy':
    # run this command for debugging.
    print(aci_service.get_logs())
    aci_service.delete()

## Success!

If you've made it this far, you've deployed a working web service that does object detection using an ONNX model. You can get the URL for the webservice with the code below.

In [ ]:
print(aci_service.scoring_uri)

When you are eventually done using the web service, remember to delete it.

In [ ]:
#aci_service.delete()